In [1]:
import xarray as xr
from bokeh.plotting import output_file, save, output_notebook
from core_portrait_plot import portrait_plot
import numpy as np

In [2]:
# Enable Bokeh output in the notebook
output_notebook()

Loading BokehJS ...

In [3]:
mdl_list = ['CanESM5', 'CMCC-CM2-SR5', 'CNRM-ESM2-1', 'EC-Earth3', 'FGOALS-f3-L', 'HadGEM3-GC31-MM', 'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorCPM1']

filename1 = "/global/cfs/projectdirs/m4581/jungchoi/PMP/GMD2026/Fig1/Fig1.Mean_bias.TAS.nc"
filename2 = "/global/cfs/projectdirs/m4581/jungchoi/PMP/GMD2026/Fig1/Fig1.Mean_bias.PR.nc"

ds1 = xr.open_dataset(filename1)
ds2 = xr.open_dataset(filename2)

var1_data = ds1["tas"].values
var2_data = ds2["pr"].values

In [4]:
arctic_index = np.arange(0, 55, 5)
nh_index = np.arange(1, 55, 5)
tropics_index = np.arange(2, 55, 5)
sh_index = np.arange(3, 55, 5)
antarctic_index = np.arange(4, 55, 5)

In [5]:
arctic1_data = []
for a in arctic_index:
    arctic1_data.append(var1_data[a])

nh1_data = []
for nh in nh_index:
    nh1_data.append(var1_data[nh])
    
tropics1_data = []
for t in tropics_index:
    tropics1_data.append(var1_data[t])
    
sh1_data = []
for sh in sh_index:
    sh1_data.append(var1_data[sh])
    
antarctic1_data = []
for an in antarctic_index:
    antarctic1_data.append(var1_data[an])

In [6]:
xaxis_labels = ['LY1', 'LY2', 'LY3', 'LY4', 'LY5', 'LY6', 'LY7', 'LY8', 'LY9', 'LY10', 'HIST']
yaxis_labels = mdl_list

In [7]:
p = portrait_plot(
    [arctic1_data, nh1_data, tropics1_data, sh1_data, antarctic1_data][::-1], 
    static=True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="RdBu_r",
    vrange=(-4,4),
    #cmap_bounds=[-4, -3.5, -3, -2.5, -2, -1.5, -1, -.5, 0, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4],
    title="TAS Mean Bias (ref: ERA5 1981-2010)",
    show_plot=False
    )

In [8]:
# Adding some customizations
from bokeh.plotting import figure, show
from bokeh.models import Span, Title, Div
from bokeh.io import output_file, save
from bokeh.layouts import column, row

line_locations = np.arange(0, 12, 1)

for loc in line_locations:
    span = Span(location=loc, dimension='width', line_color='black', line_width=2)
    p.add_layout(span)
    
for loc in line_locations:
    span = Span(location=loc, dimension='height', line_color='black', line_width=0.5)
    p.add_layout(span)
    
legend_title = Title(text="[K]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

#show(p)

#output_file(filename="static_TAS_mean_bias_portrait_plot.html", title="Static TAS Mean Bias Portrait Plot")
#save(p, filename='./static_TAS_mean_bias_portrait_plot.html')

In [9]:
# custom function that creates an array of img URLs for each glyph, with custom sort
# use xaxis labels and mdl_list
def get_img_urls(var: str='PR'):
    urls = []
    url_head = f'https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/Mean_bias.{var}'
    for n in range(5):
        for mdl in mdl_list[::-1]:
            for lbl in xaxis_labels:
                url = f'{url_head}.{mdl}.{lbl}.png'
                urls.append(url)
    return urls
    

In [10]:
pr_img_urls = get_img_urls(var='PR')
tas_img_urls = get_img_urls(var='TAS')

In [11]:
# creating interactive version
p = portrait_plot(
    [arctic1_data, nh1_data, tropics1_data, sh1_data, antarctic1_data][::-1],
    img_url = tas_img_urls,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="RdBu_r",
    vrange=(-4,4),
    #cmap_bounds=[-4, -3.5, -3, -2.5, -2, -1.5, -1, -.5, 0, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4],
    title="TAS Mean Bias (ref: ERA5 1981-2010)",
    show_plot=False
    )

line_locations = np.arange(0, 12, 1)

for loc in line_locations:
    h_lines = Span(location=loc, dimension='width', line_color='black', line_width=3)
    p.add_layout(h_lines)
    
for loc in line_locations:
    v_lines = Span(location=loc, dimension='height', line_color='black', line_width=0.7)
    p.add_layout(v_lines)
    
end_line = Span(location=10.99, dimension='height', line_color='black', line_width=0.7)
p.add_layout(end_line)

hist_line = Span(location=10, dimension='height', line_color='black', line_width=3)
p.add_layout(hist_line)
    
legend_title = Title(text="[K]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

# Add text to top of the page
text_title = """
<div class=title>
<center>
<h1>CMIP6 DCPP TAS Mean bias</h1>
</center>
</div>
"""
logos = """
<div class=title>
<center>
<br>

<a href="https://pcmdi.llnl.gov/" target="_blank">
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_PCMDI.png" height="50" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="PCMDI"
    title="PCMDI"></a>

<a href=https://climatemodeling.science.energy.gov/program/regional-global-model-analysis target=_blank>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_doe.png height="45" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="US Sponsor: DOE BER RGMA"
    title="US Sponsor: DOE BER RGMA"></a>

</center>
</div>
<br>
"""

text_top = text_title + logos
text_top += """
<style>
.title {
    width: 160%;
}
.box1 {
    background-color: #E8E8E8;
    width: 160%;
    padding: 10px;
    margin: 0px;
}
.dropdown {
    background-color: white;
    width: 100%;
    padding: 10px;
    margin: 0px;
}
</style>

<div class=box1>
<p><b><font size=2 color=#e67e22>Supported browsers: Chrome, Firefox, Safari, Microsoft Edge</font></b></p>

<p><b><font size=4 color=darkblue> Reference and Contributions </font></b><br>
- Citation: Choi et al. (2026, in prep.)<br>
- The calculation has been conducted and dive down plots were generated by Jung Choi.<br>
- The Interactive Portrait Plot and webpage are generated by <a href=https://people.llnl.gov/chang61 target=_blank>Kristin Chang</a> and <a href=https://pcmdi.llnl.gov/staff/lee/ target=_blank>Jiwoo Lee</a> (PCMDI/LLNL).<br>
"""

overall_text_usage = """
<p><b><font size=4 color=darkblue> Usage</b> <i>(Important for navigating to dive-down diagnostics)</i></font><br>
- Hover your mouse over boxes will show tooltips for metric values and preview of dive down plots.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/tap.svg style="height: 15px;">: Turn on/off <b>Tap</b> icon to enable/disable clicking on boxes.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/save.svg style="height: 15px;">: Click <b>Save</b> icon to download a static HTML file of the plot.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/hover.svg style="height: 15px;">: Turn on/off <b>Hover</b> icon enable/disable showing tooltips.<br>
"""

legend_img = """
<style>
h3 {
  float:right;
  margin: 10px -360px;
}
</style>
<h3>Region Legend</h3>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/region_legend.png height="100"
    style="vertical-align:middle;horizontal-align:right;margin:30px -400px;float:right;"
    alt="PCMDI Mean Bias Region Legend"
    title="PCMDI Mean Bias Region Legend">
"""

text_top += overall_text_usage + "</div>" + legend_img

# LLNL Disclaimer
#

text_bottom = """
<style>
.bottom {
    width: 1000px;
}
.fig_desc {
    width:1000px;
    font-size:11pt;
}
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}
td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
    text-align: center;
}
tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
"""

text_bottom += """
<div class=fig_desc>
<p><b><font size=4 color=darkblue> Figure description</b></font><br>
This portrait plot shows the mean bias in surface air temperature (TAS) for each model over the period 1981-2010. The mean bias is calculated as a function of lead year (LY) over the five subregions: Arctic (60°N-90°N), the Northern Hemisphere mid-latitudes (30°N-60°N), the Tropics (30°S-30°N), the Southern Hemisphere mid-latitudes (30°S-60°S), and Antarctic (60°S-90°S). The rightmost column indicates the mean bias from historical experiments (HIST) for the same period. ERA5 is used as the reference dataset. Units are [K]. The mean bias is calculated for each month and each grid cell, and then annual and area averages are taken. Interactive subplots show the global map of mean biases, which are produced by averaging data over two-month periods.
</div>
<br><br>
<div class=bottom>
<br><p>
<a href="https://www.llnl.gov" target=_blank>
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_llnl-600.png" 
alt="LLNL" style="float:left;height:80px;margin-right:10px">
</a>
The work of PCMDI and results presented on this web page are supported by the <a href=https://eesm.science.energy.gov/program-area/regional-global-model-analysis>Regional and Global Model Analysis (RGMA) program area
</a> under the <a href=https://eesm.science.energy.gov/about>Earth and Environmental System Modeling (EESM) program
</a> within the <a href=https://science.osti.gov/ber/Research/eessd>Earth and Environmental Systems Sciences Division (EESSD)
</a> of the <a href=https://science.osti.gov/>United States Department of Energy's (DOE) Office of Science (OSTI)
</a>. The work of PCMDI is performed under the auspices of the U.S. DOE by <a href=https://pls.llnl.gov/>Lawrence Livermore National Laboratory (LLNL)
</a> under contract DE-AC52-07NA27344. LLNL-WEB-812310.</p>
</div>
"""

# combine page
page = column(Div(text=text_top),
              p,
              Div(text=text_bottom)) 
    
#show(p)
output_file(filename="interactive_TAS_mean_bias_portrait_plot.html", title="Interactive TAS Mean Bias Portrait Plot")
save(page, filename='./interactive_TAS_mean_bias_portrait_plot.html')

'/global/cfs/cdirs/m4581/jungchoi/PMP/interactive_Mean_bias_portrait/interactive_TAS_mean_bias_portrait_plot.html'

In [12]:
# Repeat process for 2nd var
arctic2_data = []
for a in arctic_index:
    arctic2_data.append(var2_data[a])

nh2_data = []
for nh in nh_index:
    nh2_data.append(var2_data[nh])
    
tropics2_data = []
for t in tropics_index:
    tropics2_data.append(var2_data[t])
    
sh2_data = []
for sh in sh_index:
    sh2_data.append(var2_data[sh])
    
antarctic2_data = []
for an in antarctic_index:
    antarctic2_data.append(var2_data[an])

In [13]:
p = portrait_plot(
    [arctic2_data, nh2_data, tropics2_data, sh2_data, antarctic2_data][::-1], 
    img_url = pr_img_urls,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="BrBG",
    vrange=(-0.8,0.8),
    title="PR Mean Bias (ref: GPCP 1981-2010)",
    show_plot=False
    )

line_locations = np.arange(0, 12, 1)

for loc in line_locations:
    span = Span(location=loc, dimension='width', line_color='black', line_width=2)
    p.add_layout(span)
    
for loc in line_locations:
    span = Span(location=loc, dimension='height', line_color='black', line_width=0.5)
    p.add_layout(span)
    
legend_title = Title(text="[mm•d⁻¹]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'
    
#show(p)
#output_file(filename="interactive_PR_mean_bias_portrait_plot.html", title="Interactive PR Mean Bias Portrait Plot")
#save(p, filename='./interactive_PR_mean_bias_portrait_plot.html')

In [14]:
# interactive plot
p = portrait_plot(
    [arctic2_data, nh2_data, tropics2_data, sh2_data, antarctic2_data][::-1], 
    img_url = pr_img_urls,
    clickable = True,
    xaxis_labels=xaxis_labels,
    yaxis_labels=yaxis_labels,
    xaxis_fontsize=13,
    yaxis_fontsize=13,
    height=1000, width=1000,
    cmap="BrBG",
    vrange=(-0.8,0.8),
    title="PR Mean Bias (ref: GPCP 1981-2010)",
    show_plot=False
    )

line_locations = np.arange(0, 12, 1)

for loc in line_locations:
    h_lines = Span(location=loc, dimension='width', line_color='black', line_width=3)
    p.add_layout(h_lines)
    
for loc in line_locations:
    v_lines = Span(location=loc, dimension='height', line_color='black', line_width=0.7)
    p.add_layout(v_lines)
    
end_line = Span(location=10.99, dimension='height', line_color='black', line_width=0.7)
p.add_layout(end_line)

hist_line = Span(location=10, dimension='height', line_color='black', line_width=3)
p.add_layout(hist_line)
    
legend_title = Title(text="[mm•d⁻¹]", align="center", text_font_size="13pt")
p.add_layout(legend_title, 'right')
p.title.text_font_size = '14pt'

# Add text to top of the page
text_title = """
<div class=title>
<center>
<h1>CMIP6 DCPP PR Mean bias</h1>
</center>
</div>
"""
logos = """
<div class=title>
<center>
<br>

<a href="https://pcmdi.llnl.gov/" target="_blank">
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_PCMDI.png" height="50" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="PCMDI"
    title="PCMDI"></a>

<a href=https://climatemodeling.science.energy.gov/program/regional-global-model-analysis target=_blank>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_doe.png height="45" 
    style="vertical-align:middle;margin:0px 10px" 
    alt="US Sponsor: DOE BER RGMA"
    title="US Sponsor: DOE BER RGMA"></a>

</center>
</div>
<br>
"""

text_top = text_title + logos
text_top += """
<style>
.title {
    width: 160%;
}
.box1 {
    background-color: #E8E8E8;
    width: 160%;
    padding: 10px;
    margin: 0px;
}
.dropdown {
    background-color: white;
    width: 100%;
    padding: 10px;
    margin: 0px;
}
</style>

<div class=box1>
<p><b><font size=2 color=#e67e22>Supported browsers: Chrome, Firefox, Safari, Microsoft Edge</font></b></p>

<p><b><font size=4 color=darkblue> Reference and Contributions </font></b><br>
- Citation: Choi et al. (2026, in prep.)<br>
- The calculation has been conducted and dive down plots were generated by Jung Choi.<br>
- The Interactive Portrait Plot and webpage are generated by <a href=https://people.llnl.gov/chang61 target=_blank>Kristin Chang</a> and <a href=https://pcmdi.llnl.gov/staff/lee/ target=_blank>Jiwoo Lee</a> (PCMDI/LLNL).<br>
"""

overall_text_usage = """
<p><b><font size=4 color=darkblue> Usage</b> <i>(Important for navigating to dive-down diagnostics)</i></font><br>
- Hover your mouse over boxes will show tooltips for metric values and preview of dive down plots.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/tap.svg style="height: 15px;">: Turn on/off <b>Tap</b> icon to enable/disable clicking on boxes.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/save.svg style="height: 15px;">: Click <b>Save</b> icon to download a static HTML file of the plot.<br>
- <img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/bokeh_icons/hover.svg style="height: 15px;">: Turn on/off <b>Hover</b> icon enable/disable showing tooltips.<br>
"""

legend_img = """
<style>
h3 {
  float:right;
  margin: 10px -360px;
}
</style>
<h3>Region Legend</h3>
<img src=https://pcmdi.llnl.gov/pmp-preliminary-results/graphics/dcpp/mean_bias/region_legend.png height="100"
    style="vertical-align:middle;horizontal-align:right;margin:30px -400px;float:right;"
    alt="PCMDI Mean Bias Region Legend"
    title="PCMDI Mean Bias Region Legend">
"""

text_top += overall_text_usage + "</div>" + legend_img

# LLNL Disclaimer
#

text_bottom = """
<style>
.bottom {
    width: 1000px;
}
.fig_desc {
    width:1000px;
    font-size:11pt;
}
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}
td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
    text-align: center;
}
tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
"""

text_bottom += """
<div class=fig_desc>
<p><b><font size=4 color=darkblue> Figure description</b></font><br>
<p> This portrait plot shows the mean bias in precipitation (PR) for each model over the period 1981-2010. The mean bias is calculated as a function of lead year (LY) over the five subregions: Arctic (60°N-90°N), the Northern Hemisphere mid-latitudes (30°N-60°N), the Tropics (30°S-30°N), the Southern Hemisphere mid-latitudes (30°S-60°S), and Antarctic (60°S-90°S). The rightmost column indicates the mean bias from historical experiments (HIST) for the same period. GPCP is used as the reference dataset. Units are [mm d-1]. The mean bias is calculated for each month and each grid cell, and then annual and area averages are taken. Interactive subplots show the global map of mean biases, which are produced by averaging data over two-month periods.</p>
</div>
<br><br>
<div class=bottom>
<br><p>
<a href="https://www.llnl.gov" target=_blank>
<img src="https://pcmdi.llnl.gov/pmp-preliminary-results/interactive_plot/portrait_plot/enso_metric/logo/logo_llnl-600.png" 
alt="LLNL" style="float:left;height:80px;margin-right:10px">
</a>
The work of PCMDI and results presented on this web page are supported by the <a href=https://eesm.science.energy.gov/program-area/regional-global-model-analysis>Regional and Global Model Analysis (RGMA) program area
</a> under the <a href=https://eesm.science.energy.gov/about>Earth and Environmental System Modeling (EESM) program
</a> within the <a href=https://science.osti.gov/ber/Research/eessd>Earth and Environmental Systems Sciences Division (EESSD)
</a> of the <a href=https://science.osti.gov/>United States Department of Energy's (DOE) Office of Science (OSTI)
</a>. The work of PCMDI is performed under the auspices of the U.S. DOE by <a href=https://pls.llnl.gov/>Lawrence Livermore National Laboratory (LLNL)
</a> under contract DE-AC52-07NA27344. LLNL-WEB-812310.</p>
</div>
"""

page = column(Div(text=text_top),
              p,
              Div(text=text_bottom))    

#show(p)
output_file(filename="interactive_PR_mean_bias_portrait_plot.html", title="Interactive PR Mean Bias Portrait Plot")
save(page, filename='./interactive_PR_mean_bias_portrait_plot.html')

'/global/cfs/cdirs/m4581/jungchoi/PMP/interactive_Mean_bias_portrait/interactive_PR_mean_bias_portrait_plot.html'